In [ ]:
# Load trained model
import gradio as gr
from sahi.predict import get_sliced_prediction
from sahi import AutoDetectionModel
from ultralytics import YOLO
models = 'your_path_model' #ganti path model

detection_model = AutoDetectionModel.from_pretrained(
    model_type='yolov8',
    model_path=models,
    confidence_threshold=0.3,
    device="cuda:0"  # Gunakan GPU jika tersedia
)

# Function untuk inference dengan Gradio
def detect_objects(image):
    # Lakukan prediksi dengan slicing
    result = get_sliced_prediction(
        image,
        detection_model,
        slice_height=256,
        slice_width=256,
        overlap_height_ratio=0.2,
        overlap_width_ratio=0.2,
    )

    # Simpan hasil visualisasi
    output_path = "output.jpg"
    result.export_visuals(export_dir=".", text_size=0.5, hide_labels=False, rect_th=2)
    
    return output_path, len(result.object_prediction_list)

# Buat antarmuka Gradio
interface = gr.Interface(
    fn=detect_objects,
    inputs=gr.Image(type="filepath"),
    outputs=[gr.Image(label="Detection Result"), gr.Number(label="Object Count")],
    title="YOLOv8 Object Detection with SAHI",
    description="Upload an image and detect objects using a YOLOv8 model with slicing.",
)

# Jalankan aplikasi
interface.launch(debug=True, share=True)